# Temperature and Humidity Prediction

This notebook implements regression models for temperature and humidity prediction using other air quality features.

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

## 1. Data Preparation

In [ ]:
# Load and prepare data
df = pd.read_csv('data.csv')

# Select features for prediction (excluding temperature and humidity)
reg_features = ['CO2', 'TVOC', 'PM10', 'PM2.5', 'CO', 'LDR', 'O3']
X_reg = df[reg_features]
y_temp = df['Temp']
y_hum = df['Hum']

# Split data
X_train_reg, X_test_reg, y_train_temp, y_test_temp = train_test_split(X_reg, y_temp, test_size=0.2, random_state=42)
_, _, y_train_hum, y_test_hum = train_test_split(X_reg, y_hum, test_size=0.2, random_state=42)

# Scale features
scaler_reg = StandardScaler()
X_train_reg_scaled = scaler_reg.fit_transform(X_train_reg)
X_test_reg_scaled = scaler_reg.transform(X_test_reg)

print("Data shapes:")
print(f"Training features: {X_train_reg_scaled.shape}")
print(f"Test features: {X_test_reg_scaled.shape}")

## 2. Temperature Prediction Model

In [ ]:
def create_regression_model(input_dim):
    model = Sequential([
        Dense(128, activation='relu', input_shape=(input_dim,)),
        Dense(64, activation='relu'),
        Dense(32, activation='relu'),
        Dense(1)
    ])
    return model

# Create and train temperature model
print("Training Temperature Prediction Model...")
temp_model = create_regression_model(len(reg_features))
temp_model.compile(optimizer='adam', loss='mse')

temp_history = temp_model.fit(
    X_train_reg_scaled,
    y_train_temp,
    epochs=50,
    validation_split=0.2,
    verbose=1
)

# Make predictions
temp_pred = temp_model.predict(X_test_reg_scaled).flatten()

# Calculate metrics
temp_metrics = {
    'MSE': mean_squared_error(y_test_temp, temp_pred),
    'RMSE': np.sqrt(mean_squared_error(y_test_temp, temp_pred)),
    'R²': r2_score(y_test_temp, temp_pred),
    'MAE': mean_absolute_error(y_test_temp, temp_pred)
}

print("\nTemperature Prediction Metrics:")
for metric, value in temp_metrics.items():
    print(f"{metric}: {value:.4f}")

## 3. Humidity Prediction Model

In [ ]:
# Create and train humidity model
print("Training Humidity Prediction Model...")
hum_model = create_regression_model(len(reg_features))
hum_model.compile(optimizer='adam', loss='mse')

hum_history = hum_model.fit(
    X_train_reg_scaled,
    y_train_hum,
    epochs=50,
    validation_split=0.2,
    verbose=1
)

# Make predictions
hum_pred = hum_model.predict(X_test_reg_scaled).flatten()

# Calculate metrics
hum_metrics = {
    'MSE': mean_squared_error(y_test_hum, hum_pred),
    'RMSE': np.sqrt(mean_squared_error(y_test_hum, hum_pred)),
    'R²': r2_score(y_test_hum, hum_pred),
    'MAE': mean_absolute_error(y_test_hum, hum_pred)
}

print("\nHumidity Prediction Metrics:")
for metric, value in hum_metrics.items():
    print(f"{metric}: {value:.4f}")

## 4. Visualization of Results

In [ ]:
# Plot training history
plt.figure(figsize=(15, 5))

# Temperature training history
plt.subplot(1, 2, 1)
plt.plot(temp_history.history['loss'], label='Training')
plt.plot(temp_history.history['val_loss'], label='Validation')
plt.title('Temperature Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Mean Squared Error')
plt.legend()

# Humidity training history
plt.subplot(1, 2, 2)
plt.plot(hum_history.history['loss'], label='Training')
plt.plot(hum_history.history['val_loss'], label='Validation')
plt.title('Humidity Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Mean Squared Error')
plt.legend()

plt.tight_layout()
plt.show()

# Plot predictions vs actual values
plt.figure(figsize=(15, 5))

# Temperature predictions
plt.subplot(1, 2, 1)
plt.scatter(y_test_temp, temp_pred, alpha=0.5)
plt.plot([y_test_temp.min(), y_test_temp.max()],
         [y_test_temp.min(), y_test_temp.max()], 'r--')
plt.title(f'Temperature Prediction (R² = {temp_metrics["R²"]:.4f})')
plt.xlabel('Actual Temperature')
plt.ylabel('Predicted Temperature')

# Humidity predictions
plt.subplot(1, 2, 2)
plt.scatter(y_test_hum, hum_pred, alpha=0.5)
plt.plot([y_test_hum.min(), y_test_hum.max()],
         [y_test_hum.min(), y_test_hum.max()], 'r--')
plt.title(f'Humidity Prediction (R² = {hum_metrics["R²"]:.4f})')
plt.xlabel('Actual Humidity')
plt.ylabel('Predicted Humidity')

plt.tight_layout()
plt.show()

## 5. Save Results

In [ ]:
# Create models directory
os.makedirs('models', exist_ok=True)

# Save models
temp_model.save('models/temperature_model.h5')
hum_model.save('models/humidity_model.h5')

# Save metrics
metrics = {
    'Temperature': temp_metrics,
    'Humidity': hum_metrics
}

import pickle
with open('results/regression_metrics.pkl', 'wb') as f:
    pickle.dump(metrics, f)

print("Models and metrics saved successfully!")